<a href="https://colab.research.google.com/github/caixahub/Barsi_Project/blob/main/Carteira_do_Barsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as web
import seaborn as sns
import yfinance as yf
import tensorflow as tf
from tensorflow import keras

from datetime import datetime,date

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.metrics import mean_squared_error
from keras.layers  import LSTM

# Needed to help our plots look cleaner with plotly

import plotly.graph_objects as go
import datetime
from datetime import datetime
from plotly.offline import init_notebook_mode, iplot
from plotly import tools

In [45]:
tickers = ['BBAS3.SA', 'IRBR3.SA', 'KLBN11.SA', 'SUZB3.SA', 'UNIP6.SA', 'TASA4.SA', 'TAEE11.SA']
df = yf.download(tickers, start = '2017-01-01', period= '1d')

[*********************100%***********************]  7 of 7 completed


In [46]:
#check the dimensions of the data
df.shape

(1558, 42)

In [47]:
#view the first 5 rows of the data
df.head()

Adj Close                                                      \
             BBAS3.SA IRBR3.SA  KLBN11.SA   SUZB3.SA  TAEE11.SA  TASA4.SA   
Date                                                                        
2017-01-02  19.448151      NaN  13.230025  18.122595  11.511273  1.716108   
2017-01-03  20.337934      NaN  13.562712  18.122595  11.532791  1.744241   
2017-01-04  20.231998      NaN  13.059813  18.122595  11.430590  1.631709   
2017-01-05  20.182568      NaN  12.943762  18.122595  11.484375  1.687975   
2017-01-06  20.401491      NaN  12.881865  18.122595  11.387554  1.641087   

                          Close                      ...       Open           \
            UNIP6.SA   BBAS3.SA IRBR3.SA  KLBN11.SA  ...  TAEE11.SA TASA4.SA   
Date                                                 ...                       
2017-01-02  1.991450  27.540001      NaN  17.100000  ...  20.709999     1.75   
2017-01-03  1.968124  28.799999      NaN  17.530001  ...  21.430000     1.86   
2017-01-04  1.988535  28.650000      NaN  16.879999  ...  21.410000     1.81   
2017-01-05  1.997282  28.580000      NaN  16.730000  ...  21.209999     1.78   
2017-01-06  1.976872  28.889999      NaN  16.650000  ...  21.260000     1.78   

                       Volume                                          \
            UNIP6.SA BBAS3.SA IRBR3.SA  KLBN11.SA SUZB3.SA  TAEE11.SA   
Date                                                                    
2017-01-02  4.885713  1968200      NaN   841000.0        0   572300.0   
2017-01-03  4.835713  7578900      NaN  2080700.0        0  1292700.0   
2017-01-04  4.885713  4156300      NaN  1805000.0        0  1173400.0   
2017-01-05  4.885713  5457100      NaN  1837400.0        0  1069600.0   
2017-01-06  4.807141  3692400      NaN  1356700.0        0   574100.0   

                              
           TASA4.SA UNIP6.SA  
Date                          
2017-01-02    15200     2660  
2017-01-03    11500    45360  
2017-01-04    56900     1400  
2017-01-05    14900    48860  
2017-01-06     1000     1820  

[5 rows x 42 columns]

In [48]:
#view the last 5 values of the data
df.tail()

Adj Close                                                         \
             BBAS3.SA   IRBR3.SA  KLBN11.SA   SUZB3.SA  TAEE11.SA   TASA4.SA   
Date                                                                           
2023-03-29  38.310001  21.129999  18.180000  43.070000  34.919998  15.500000   
2023-03-30  39.009998  22.200001  18.260000  43.049999  34.849998  16.379999   
2023-03-31  39.110001  22.240000  18.090000  41.599998  34.830002  16.180000   
2023-04-03  38.650002  21.350000  18.230000  42.740002  34.650002  16.389999   
2023-04-04  39.290001  21.980000  18.309999  42.349998  35.180000  16.270000   

                           Close                        ...       Open  \
             UNIP6.SA   BBAS3.SA   IRBR3.SA  KLBN11.SA  ...  TAEE11.SA   
Date                                                    ...              
2023-03-29  71.639999  38.310001  21.129999  18.180000  ...  35.230000   
2023-03-30  71.089996  39.009998  22.200001  18.260000  ...  35.090000   
2023-03-31  70.050003  39.110001  22.240000  18.090000  ...  34.980000   
2023-04-03  69.250000  38.650002  21.350000  18.230000  ...  34.849998   
2023-04-04  69.970001  39.290001  21.980000  18.309999  ...  35.000000   

                                    Volume                                  \
             TASA4.SA   UNIP6.SA  BBAS3.SA   IRBR3.SA  KLBN11.SA  SUZB3.SA   
Date                                                                         
2023-03-29  15.450000  73.660004   7197800  1063200.0  5026000.0   3955700   
2023-03-30  15.600000  71.660004  13147600  1717500.0  5087700.0   5943600   
2023-03-31  16.389999  71.559998   8450100  1633100.0  6252600.0  14261800   
2023-04-03  16.150000  70.730003   8960300  1382600.0  4081800.0   6884200   
2023-04-04  16.440001  69.650002   8219000  1602300.0  5157000.0   4730300   

                                         
            TAEE11.SA TASA4.SA UNIP6.SA  
Date                                     
2023-03-29  2225500.0   589300   230100  
2023-03-30  2522900.0   806500   254100  
2023-03-31  2057300.0  1065700   406100  
2023-04-03  2448600.0   642800   221700  
2023-04-04  3813500.0   514700   200300  

[5 rows x 42 columns]

In [49]:
# #check if there are missing values for each type of stock

df.isnull().sum()

Adj Close  BBAS3.SA       0
           IRBR3.SA     144
           KLBN11.SA      1
           SUZB3.SA       0
           TAEE11.SA      1
           TASA4.SA       0
           UNIP6.SA       0
Close      BBAS3.SA       0
           IRBR3.SA     144
           KLBN11.SA      1
           SUZB3.SA       0
           TAEE11.SA      1
           TASA4.SA       0
           UNIP6.SA       0
High       BBAS3.SA       0
           IRBR3.SA     144
           KLBN11.SA      1
           SUZB3.SA       0
           TAEE11.SA      1
           TASA4.SA       0
           UNIP6.SA       0
Low        BBAS3.SA       0
           IRBR3.SA     144
           KLBN11.SA      1
           SUZB3.SA       0
           TAEE11.SA      1
           TASA4.SA       0
           UNIP6.SA       0
Open       BBAS3.SA       0
           IRBR3.SA     144
           KLBN11.SA      1
           SUZB3.SA       0
           TAEE11.SA      1
           TASA4.SA       0
           UNIP6.SA       0
Volume     BBAS3.SA 

In [50]:
# some missing values, and we will use the fillna method to resolve the missing values.

# handing missing values
df.fillna(method='ffill', inplace = True) # use front fill method
df.fillna(method='bfill', inplace = True) # use back fill method
#check to see if there are still any missing values
df.isnull().sum()

Adj Close  BBAS3.SA     0
           IRBR3.SA     0
           KLBN11.SA    0
           SUZB3.SA     0
           TAEE11.SA    0
           TASA4.SA     0
           UNIP6.SA     0
Close      BBAS3.SA     0
           IRBR3.SA     0
           KLBN11.SA    0
           SUZB3.SA     0
           TAEE11.SA    0
           TASA4.SA     0
           UNIP6.SA     0
High       BBAS3.SA     0
           IRBR3.SA     0
           KLBN11.SA    0
           SUZB3.SA     0
           TAEE11.SA    0
           TASA4.SA     0
           UNIP6.SA     0
Low        BBAS3.SA     0
           IRBR3.SA     0
           KLBN11.SA    0
           SUZB3.SA     0
           TAEE11.SA    0
           TASA4.SA     0
           UNIP6.SA     0
Open       BBAS3.SA     0
           IRBR3.SA     0
           KLBN11.SA    0
           SUZB3.SA     0
           TAEE11.SA    0
           TASA4.SA     0
           UNIP6.SA     0
Volume     BBAS3.SA     0
           IRBR3.SA     0
           KLBN11.SA    0
           S

In [25]:
#view descriptive statistics of adjusted close process of the stocks

df[['Adj Close']].describe()

Adj Close                                                      \
          BBAS3.SA     IRBR3.SA    KLBN11.SA     SUZB3.SA    TAEE11.SA   
count  1558.000000  1558.000000  1558.000000  1558.000000  1558.000000   
mean     30.008892   304.475899    17.749022    39.594539    22.668656   
std       6.404701   258.272548     3.947339    13.762799     9.435612   
min      18.166697    18.450001    10.714169    16.001631    11.183149   
25%      25.016490   128.938995    14.258629    30.675387    12.891095   
50%      28.693674   216.538544    17.219309    41.355585    20.502330   
75%      36.008156   416.682091    20.994646    49.215112    32.224205   
max      43.551609  1030.962036    27.867048    74.390305    40.624340   

                                 
          TASA4.SA     UNIP6.SA  
count  1558.000000  1558.000000  
mean      9.369379    36.254399  
std       8.002968    30.104588  
min       1.584821     1.965209  
25%       2.363165    16.065682  
50%       5.087368    21.225051  
75%      16.962500    66.940725  
max      26.970087   109.632782

In [26]:
# view general info
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1558 entries, 2017-01-02 to 2023-04-04
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   (Adj Close, BBAS3.SA)   1558 non-null   float64
 1   (Adj Close, IRBR3.SA)   1558 non-null   float64
 2   (Adj Close, KLBN11.SA)  1558 non-null   float64
 3   (Adj Close, SUZB3.SA)   1558 non-null   float64
 4   (Adj Close, TAEE11.SA)  1558 non-null   float64
 5   (Adj Close, TASA4.SA)   1558 non-null   float64
 6   (Adj Close, UNIP6.SA)   1558 non-null   float64
 7   (Close, BBAS3.SA)       1558 non-null   float64
 8   (Close, IRBR3.SA)       1558 non-null   float64
 9   (Close, KLBN11.SA)      1558 non-null   float64
 10  (Close, SUZB3.SA)       1558 non-null   float64
 11  (Close, TAEE11.SA)      1558 non-null   float64
 12  (Close, TASA4.SA)       1558 non-null   float64
 13  (Close, UNIP6.SA)       1558 non-null   float64
 14  (High, BBAS3.SA)      

In [27]:
# View the maximum close date of stocks
def max_close(stocks,df):
    """ This calculates and returns the maximum closing value of a specific stock"""
    return df['Close'][stocks].max() # computes and returns the maximum closing stock value

In [28]:
# test the above function with specific stocks
def test_max():
    """ This tests the max_close function"""
    for stocks in ["SUZB3.SA", "BBAS3.SA", "IRBR3.SA", "KLBN11.SA", "TAEE11.SA", "TASA4.SA", "UNIP6.SA"]:
        print("Maxiumum Closing Value for {} is {}".format(stocks, max_close(stocks,df)))

test_max() 

Maxiumum Closing Value for SUZB3.SA is 79.18000030517578
Maxiumum Closing Value for BBAS3.SA is 55.18000030517578
Maxiumum Closing Value for IRBR3.SA is 1032.82470703125
Maxiumum Closing Value for KLBN11.SA is 31.239999771118164
Maxiumum Closing Value for TAEE11.SA is 45.75
Maxiumum Closing Value for TASA4.SA is 28.760000228881836
Maxiumum Closing Value for UNIP6.SA is 114.83999633789062


In [39]:
# Function that get specific stock data and fills in any missing value
def get_data(df, stocks):
    """
    This function gets a specific stock data and fills in any missing values using the fill forward and fill backward methods
    Input: 
    df - dataframe
    stocks - the type of stock
    Output - a cleaned dataset to be used for prediction
    """
    df1 = pd.DataFrame (data = df.iloc[:, df.columns.get_level_values(1)==stocks].values,
                          index = df.iloc[:, df.columns.get_level_values(1)==stocks].index,
                          columns = df.iloc[:, df.columns.get_level_values(1)==stocks].columns.get_level_values(0))
    
    df1.fillna(method='ffill', inplace= True)
    df1.fillna(method='bfill', inplace=True)
    
    return df1

In [14]:
acao = get_data(df, 'UNIP6.SA')  
acao['mm_20p'] = acao.Close.rolling(20).mean()
acao['max_mm_ant'] = acao.mm_20p.rolling(2, closed='left' ).max()
acao['min_mm_ant'] = acao.mm_20p.rolling(2, closed='left' ).min()
acao['cond_alta'] = (acao.Close > acao.mm_20p) & (acao.mm_20p > acao.max_mm_ant)
acao['cond_baixa'] = (acao.Close < acao.mm_20p) & (acao.mm_20p < acao.min_mm_ant)


In [15]:

fig = go.Figure(data=[go.Candlestick(x=acao.index,
                
                open=acao['Open'],
                high=acao['High'],
                low=acao['Low'],
                close=acao['Close'])])

fig.add_trace(go.Scatter(name='Sma_20', x=acao.index, y=acao['mm_20p'], marker_color='gray'))
fig.update_layout(template='ggplot2', plot_bgcolor='black' , width=1200, height=800)
fig.show()